# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
import numpy as np
from collections import OrderedDict
import random
from PIL import Image

from torchvision import datasets
from torchvision import transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [2]:
data_dir = 'flower_data/'
train_dir = data_dir + 'train/'
valid_dir = data_dir + 'valid/'

In [3]:
# TODONE: Define your transforms for the training and validation sets
augmentations = [
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),
    transforms.RandomVerticalFlip(p=0.3) ]

train_transforms = transforms.Compose([
    transforms.RandomOrder(augmentations),
    transforms.RandomResizedCrop(224, ratio=(.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

valid_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

# TODONE: Load the datasets with ImageFolder
train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)

valid_datasets = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODONE: Using the image datasets and the trainforms, define the dataloaders
batch_size = 20
num_workers = 0 # Make it a main process, loading as needed; slower, but unlikely to run out of memory

train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

valid_loader = torch.utils.data.DataLoader(valid_datasets, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [5]:
# TODONE: Build and train your network

## Models to consider

# res152 = models.resnet152(pretrained=True)  ## Ran out of memory
# print(res152)
# features = [res152.conv1, res152.bn1, res152.relu, res152.maxpool, res152.layer1, res152.layer2, res152.layer3, res152.layer4, res152.avgpool]

# res50 = models.resnet50(pretrained=True)  ## Good:: Epoch: 68 - Training Loss: 0.065516 - Validation Loss: 0.342470
# print(res50)

# dense161 = models.densenet161(pretrained=True)  ## Probably will run out of memory, and indeed it did
# print(dense161)

# dense121 = models.densenet121(pretrained=True) ## Best:: Epoch: 64 - Training Loss: 0.052736 - Validation Loss: 0.168353
# print(dense121)

model = models.densenet121(pretrained=True)
print(model)

C:\Users\Watki\Anaconda3\envs\MLND\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

)


In [6]:
filename = 'deeper_dense121'

for param in model.parameters():
    param.require_grad = False
    
if filename is 'res50':
    print(model.fc.in_features, model.fc.out_features)

    n_inputs = model.fc.in_features

    last_layer = nn.Linear(n_inputs, len(cat_to_name))

    model.fc = last_layer

    print(model.fc.in_features, model.fc.out_features)

else:
    print(model.classifier.in_features, model.classifier.out_features)
    
    n_inputs = model.classifier.in_features

    # add last linear layer (n_inputs -> 5 flower classes)
    # new layers automatically have requires_grad = True
    last_layers = nn.Sequential(OrderedDict([
        ('drop1', nn.Dropout(0.1)),
        ('fc1', nn.Linear(n_inputs, 512)),
        ('relu1', nn.ReLU()),
        ('drop2', nn.Dropout(0.3)),
        ('fc2', nn.Linear(512, len(cat_to_name))) ]))
        
    model.classifier = last_layers

    # check to see that your last layer produces the expected number of outputs
    print(model)

1024 1000
DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): R

)


In [7]:
# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [ ]:
# number of epochs to train the model
n_epochs = 100

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        best_epoch = epoch
        best_loss = valid_loss
        torch.save(model.state_dict(), f'model_{filename}_{epoch}.pt')
        torch.save(optimizer.state_dict(), f'optimizer_{filename}_{epoch}.pt')
        torch.save(model, f'best_model_{filename}.pt')
        torch.save(optimizer, f'best_optimizer_{filename}.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 4.240837 	Validation Loss: 3.848322
Validation loss decreased (inf --> 3.848322).  Saving model ...
Epoch: 2 	Training Loss: 3.830423 	Validation Loss: 3.414813
Validation loss decreased (3.848322 --> 3.414813).  Saving model ...
Epoch: 3 	Training Loss: 3.387042 	Validation Loss: 2.832042
Validation loss decreased (3.414813 --> 2.832042).  Saving model ...
Epoch: 4 	Training Loss: 3.031815 	Validation Loss: 2.807809
Validation loss decreased (2.832042 --> 2.807809).  Saving model ...
Epoch: 5 	Training Loss: 2.810778 	Validation Loss: 2.035235
Validation loss decreased (2.807809 --> 2.035235).  Saving model ...
Epoch: 6 	Training Loss: 2.542110 	Validation Loss: 1.708808
Validation loss decreased (2.035235 --> 1.708808).  Saving model ...
Epoch: 7 	Training Loss: 2.339306 	Validation Loss: 1.512875
Validation loss decreased (1.708808 --> 1.512875).  Saving model ...
Epoch: 8 	Training Loss: 2.204065 	Validation Loss: 1.429649
Validation loss decreased (1.51287

### Start time 15:29

In [ ]:
model.load_state_dict(torch.load(f'model_{filename}_{best_epoch}.pt'))
optimizer.load_state_dict(torch.load(f'optimizer_{filename}_{best_epoch}.pt'))

In [8]:
model.load_state_dict(torch.load(f'model_{filename}_52.pt'))
optimizer.load_state_dict(torch.load(f'optimizer_{filename}_52.pt'))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [11]:
# TODONE: Save the checkpoint 

model.class_to_idx = train_datasets.class_to_idx

checkpoint = {
    'arch': filename,
    'class_to_idx': model.class_to_idx,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'classifier': model.classifier,
    'epoch': best_epoch,
    'best_loss': best_loss }

with open(f'checkpoint_{filename}_{best_epoch}.net', 'wb') as f:
    torch.save(checkpoint, f)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODONE: Write a function that loads a checkpoint and rebuilds the model

def checkpoint_loader(path):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    checkpoint = torch.load(path, map_location=device)
    
    model = models.densenet121(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    model.class_to_idx = checkpoint['class_to_idx']
    
    n_inputs = model.classifier.in_features
#     last_layer = nn.Linear(n_inputs, 102)
    last_layer = nn.Sequential(OrderedDict([
        ('drop1', nn.Dropout(0.1)),
        ('fc1', nn.Linear(n_inputs, 512)),
        ('relu1', nn.ReLU()),
        ('drop2', nn.Dropout(0.3)),
        ('fc2', nn.Linear(512, 102)) ]))
    
    model.classifier = last_layer
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

def add_softmax(model):
    # Adds a Softmax layer at the end of an existing model
    
    new_model = nn.Sequential(model,
                            nn.Softmax(dim=1))
    return new_model
  
# Load your model to this variable
premodel = checkpoint_loader('checkpoint_deeper_dense121_52.net')

model = add_softmax(premodel)

print(model)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODONE: Process a PIL image for use in a PyTorch model
    img = Image.open(image)
    
    if img.size[0] > img.size[1]:
        img.thumbnail((10000, 256))
    else:
        img.thumbnail((256, 10000))
    
    left_margin = (img.width-224)/2
    bottom_margin = (img.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    img = img.crop((left_margin, bottom_margin, right_margin,   
                      top_margin))
    
    img = np.array(img)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = (img - mean)/std
    
    img = img.transpose((2, 0, 1))
    
    return img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODONE: Implement the code to predict the class from an image file
    img = process_image(image_path)
    

    image_tensor = torch.from_numpy(img).type(torch.FloatTensor)

    model_input = image_tensor.unsqueeze(0)
    
    if train_on_gpu:
        image_tensor.to('cpu')
        model_input.to('cpu')
        model.to('cpu')
    
    probabilities = torch.exp(model.forward(model_input))
    
    top_probabilities, top_labels = probabilities.topk(top_num)
    top_probabilities = top_probabilities.detach().numpy().tolist()[0] 
    top_labels = top_labels.detach().numpy().tolist()[0]


    idx_to_class = {value: key for key, value in    
                                      model.class_to_idx.items()}
    
    top_classes = [idx_to_class[lab] for lab in top_labs]
    top_flowers = [cat_to_name[idx_to_class[lab]] for lab in top_labs]
    return top_probabilities, top_classes, top_flowers

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODONE: Display an image along with the top 5 classes

def plot_solution(image_path, model):

    plt.figure(figsize = (6,10))
    ax = plt.subplot(2,1,1)

    flower_num = image_path.split('/')[3]
    title_ = cat_to_name[flower_num]

    img = process_image(image_path)
    imshow(img, ax, title = title_);

    probabilities, labels, flowers = predict(image_path, model) 

    plt.subplot(2,1,2)
    sns.barplot(x=probabilities, y=flowers, color=sns.color_palette()[0]);
    plt.show()

In [ ]:
image_path = os.path.join(valid_dir, '28/image_05265.jpg')
plot_solution(image_path, model)

## Dense 121 Training Log

Epoch: 1 	Training Loss: 1.840837 	Validation Loss: 0.726808
Validation loss decreased (inf --> 0.726808).  Saving model ...
Epoch: 2 	Training Loss: 0.640676 	Validation Loss: 0.789664
Epoch: 3 	Training Loss: 0.461205 	Validation Loss: 0.471064
Validation loss decreased (0.726808 --> 0.471064).  Saving model ...
Epoch: 4 	Training Loss: 0.341942 	Validation Loss: 0.480123
Epoch: 5 	Training Loss: 0.289058 	Validation Loss: 0.351077
Validation loss decreased (0.471064 --> 0.351077).  Saving model ...
Epoch: 6 	Training Loss: 0.240028 	Validation Loss: 0.296366
Validation loss decreased (0.351077 --> 0.296366).  Saving model ...
Epoch: 7 	Training Loss: 0.220807 	Validation Loss: 0.380536
Epoch: 8 	Training Loss: 0.174053 	Validation Loss: 0.265062
Validation loss decreased (0.296366 --> 0.265062).  Saving model ...
Epoch: 9 	Training Loss: 0.164388 	Validation Loss: 0.248538
Validation loss decreased (0.265062 --> 0.248538).  Saving model ...
Epoch: 10 	Training Loss: 0.158929 	Validation Loss: 0.245665
Validation loss decreased (0.248538 --> 0.245665).  Saving model ...
Epoch: 11 	Training Loss: 0.179399 	Validation Loss: 0.253892
Epoch: 12 	Training Loss: 0.143062 	Validation Loss: 0.299004
Epoch: 13 	Training Loss: 0.132415 	Validation Loss: 0.280394
Epoch: 14 	Training Loss: 0.113570 	Validation Loss: 0.413497
Epoch: 15 	Training Loss: 0.140324 	Validation Loss: 0.217459
Validation loss decreased (0.245665 --> 0.217459).  Saving model ...
Epoch: 16 	Training Loss: 0.148951 	Validation Loss: 0.312810
Epoch: 17 	Training Loss: 0.105580 	Validation Loss: 0.312762
Epoch: 18 	Training Loss: 0.133269 	Validation Loss: 0.223173
Epoch: 19 	Training Loss: 0.099836 	Validation Loss: 0.230204
Epoch: 20 	Training Loss: 0.089988 	Validation Loss: 0.377391
Epoch: 21 	Training Loss: 0.101986 	Validation Loss: 0.207939
Validation loss decreased (0.217459 --> 0.207939).  Saving model ...
Epoch: 22 	Training Loss: 0.090497 	Validation Loss: 0.253918
Epoch: 23 	Training Loss: 0.105098 	Validation Loss: 0.257038
Epoch: 24 	Training Loss: 0.087039 	Validation Loss: 0.224589
Epoch: 25 	Training Loss: 0.075023 	Validation Loss: 0.250195
Epoch: 26 	Training Loss: 0.114490 	Validation Loss: 0.354451
Epoch: 27 	Training Loss: 0.069190 	Validation Loss: 0.313129
Epoch: 28 	Training Loss: 0.066335 	Validation Loss: 0.258219
Epoch: 29 	Training Loss: 0.078408 	Validation Loss: 0.290177
Epoch: 30 	Training Loss: 0.090112 	Validation Loss: 0.288902
Epoch: 31 	Training Loss: 0.064351 	Validation Loss: 0.257672
Epoch: 32 	Training Loss: 0.075716 	Validation Loss: 0.282621
Epoch: 33 	Training Loss: 0.062706 	Validation Loss: 0.407222
Epoch: 34 	Training Loss: 0.094904 	Validation Loss: 0.226515
Epoch: 35 	Training Loss: 0.060716 	Validation Loss: 0.292776
Epoch: 36 	Training Loss: 0.068534 	Validation Loss: 0.208206
Epoch: 37 	Training Loss: 0.068150 	Validation Loss: 0.303846
Epoch: 38 	Training Loss: 0.099623 	Validation Loss: 0.253547
Epoch: 39 	Training Loss: 0.047931 	Validation Loss: 0.226615
Epoch: 40 	Training Loss: 0.054360 	Validation Loss: 0.264230
Epoch: 41 	Training Loss: 0.072639 	Validation Loss: 0.467054
Epoch: 42 	Training Loss: 0.078949 	Validation Loss: 0.281907
Epoch: 43 	Training Loss: 0.048481 	Validation Loss: 0.318476
Epoch: 44 	Training Loss: 0.065067 	Validation Loss: 0.221030
Epoch: 45 	Training Loss: 0.047972 	Validation Loss: 0.220951
Epoch: 46 	Training Loss: 0.036499 	Validation Loss: 0.297656
Epoch: 47 	Training Loss: 0.053498 	Validation Loss: 0.216374
Epoch: 48 	Training Loss: 0.058501 	Validation Loss: 0.230687
Epoch: 49 	Training Loss: 0.092740 	Validation Loss: 0.273478
Epoch: 50 	Training Loss: 0.044600 	Validation Loss: 0.227088
Epoch: 51 	Training Loss: 0.050967 	Validation Loss: 0.321233
Epoch: 52 	Training Loss: 0.052006 	Validation Loss: 0.217489
Epoch: 53 	Training Loss: 0.030007 	Validation Loss: 0.367192
Epoch: 54 	Training Loss: 0.053345 	Validation Loss: 0.298664
Epoch: 55 	Training Loss: 0.047591 	Validation Loss: 0.300436
Epoch: 56 	Training Loss: 0.040601 	Validation Loss: 0.258214
Epoch: 57 	Training Loss: 0.055518 	Validation Loss: 0.301438
Epoch: 58 	Training Loss: 0.049291 	Validation Loss: 0.360895
Epoch: 59 	Training Loss: 0.070124 	Validation Loss: 0.227873
Epoch: 60 	Training Loss: 0.044091 	Validation Loss: 0.225992
Epoch: 61 	Training Loss: 0.027849 	Validation Loss: 0.289011
Epoch: 62 	Training Loss: 0.044952 	Validation Loss: 0.223543
Epoch: 63 	Training Loss: 0.050311 	Validation Loss: 0.260078
Epoch: 64 	Training Loss: 0.052736 	Validation Loss: 0.168353
Validation loss decreased (0.207939 --> 0.168353).  Saving model ...
Epoch: 65 	Training Loss: 0.051845 	Validation Loss: 0.257260
Epoch: 66 	Training Loss: 0.034372 	Validation Loss: 0.227684
Epoch: 67 	Training Loss: 0.033004 	Validation Loss: 0.327869
Epoch: 68 	Training Loss: 0.029804 	Validation Loss: 0.254489
Epoch: 69 	Training Loss: 0.047714 	Validation Loss: 0.282992
Epoch: 70 	Training Loss: 0.037297 	Validation Loss: 0.267831
Epoch: 71 	Training Loss: 0.069028 	Validation Loss: 0.336083
Epoch: 72 	Training Loss: 0.039041 	Validation Loss: 0.266707
Epoch: 73 	Training Loss: 0.041678 	Validation Loss: 0.235501
Epoch: 74 	Training Loss: 0.027311 	Validation Loss: 0.310032
Epoch: 75 	Training Loss: 0.044308 	Validation Loss: 0.306530
Epoch: 76 	Training Loss: 0.030147 	Validation Loss: 0.282679
Epoch: 77 	Training Loss: 0.041464 	Validation Loss: 0.333219
Epoch: 78 	Training Loss: 0.036674 	Validation Loss: 0.252494
Epoch: 79 	Training Loss: 0.031207 	Validation Loss: 0.270252
Epoch: 80 	Training Loss: 0.029693 	Validation Loss: 0.242690
Epoch: 81 	Training Loss: 0.032581 	Validation Loss: 0.365533
Epoch: 82 	Training Loss: 0.031242 	Validation Loss: 0.310142
Epoch: 83 	Training Loss: 0.050173 	Validation Loss: 0.317196
Epoch: 84 	Training Loss: 0.038698 	Validation Loss: 0.266740
Epoch: 85 	Training Loss: 0.014543 	Validation Loss: 0.249836
Epoch: 86 	Training Loss: 0.026558 	Validation Loss: 0.306637
Epoch: 87 	Training Loss: 0.029828 	Validation Loss: 0.315539
Epoch: 88 	Training Loss: 0.027777 	Validation Loss: 0.262227
Epoch: 89 	Training Loss: 0.049472 	Validation Loss: 0.286533
Epoch: 90 	Training Loss: 0.028712 	Validation Loss: 0.228773
Epoch: 91 	Training Loss: 0.022180 	Validation Loss: 0.205043
Epoch: 92 	Training Loss: 0.024710 	Validation Loss: 0.277116
Epoch: 93 	Training Loss: 0.020194 	Validation Loss: 0.247325
Epoch: 94 	Training Loss: 0.040040 	Validation Loss: 0.278157
Epoch: 95 	Training Loss: 0.055856 	Validation Loss: 0.247575
Epoch: 96 	Training Loss: 0.019991 	Validation Loss: 0.256791
Epoch: 97 	Training Loss: 0.026808 	Validation Loss: 0.344065
Epoch: 98 	Training Loss: 0.036379 	Validation Loss: 0.258369
Epoch: 99 	Training Loss: 0.025319 	Validation Loss: 0.269789
Epoch: 100 	Training Loss: 0.029979 	Validation Loss: 0.314726
Epoch: 101 	Training Loss: 0.028880 	Validation Loss: 0.290634
Epoch: 102 	Training Loss: 0.017967 	Validation Loss: 0.214559
Epoch: 103 	Training Loss: 0.024454 	Validation Loss: 0.220909
Epoch: 104 	Training Loss: 0.026005 	Validation Loss: 0.268368
Epoch: 105 	Training Loss: 0.028538 	Validation Loss: 0.253228
Epoch: 106 	Training Loss: 0.035315 	Validation Loss: 0.318982
Epoch: 107 	Training Loss: 0.031199 	Validation Loss: 0.354464
Epoch: 108 	Training Loss: 0.014863 	Validation Loss: 0.268014
Epoch: 109 	Training Loss: 0.025680 	Validation Loss: 0.514001
Epoch: 110 	Training Loss: 0.040273 	Validation Loss: 0.323925
Epoch: 111 	Training Loss: 0.035416 	Validation Loss: 0.358695
Epoch: 112 	Training Loss: 0.024871 	Validation Loss: 0.277432
Epoch: 113 	Training Loss: 0.020105 	Validation Loss: 0.221500
Epoch: 114 	Training Loss: 0.025805 	Validation Loss: 0.282289
Epoch: 115 	Training Loss: 0.032928 	Validation Loss: 0.263060
Epoch: 116 	Training Loss: 0.020915 	Validation Loss: 0.202639
Epoch: 117 	Training Loss: 0.029274 	Validation Loss: 0.283125
Epoch: 118 	Training Loss: 0.024310 	Validation Loss: 0.287807
Epoch: 119 	Training Loss: 0.010824 	Validation Loss: 0.274844
Epoch: 120 	Training Loss: 0.026688 	Validation Loss: 0.301035
Epoch: 121 	Training Loss: 0.024350 	Validation Loss: 0.265513
Epoch: 122 	Training Loss: 0.022094 	Validation Loss: 0.292715
Epoch: 123 	Training Loss: 0.025163 	Validation Loss: 0.265524
Epoch: 124 	Training Loss: 0.018339 	Validation Loss: 0.280774
Epoch: 125 	Training Loss: 0.022058 	Validation Loss: 0.293242
Epoch: 126 	Training Loss: 0.036746 	Validation Loss: 0.324282
Epoch: 127 	Training Loss: 0.018423 	Validation Loss: 0.285371
Epoch: 128 	Training Loss: 0.026406 	Validation Loss: 0.356884
Epoch: 129 	Training Loss: 0.023978 	Validation Loss: 0.336185
Epoch: 130 	Training Loss: 0.018252 	Validation Loss: 0.271707
Epoch: 131 	Training Loss: 0.009140 	Validation Loss: 0.357501
Epoch: 132 	Training Loss: 0.025322 	Validation Loss: 0.317150
Epoch: 133 	Training Loss: 0.031642 	Validation Loss: 0.384780
Epoch: 134 	Training Loss: 0.025189 	Validation Loss: 0.292454
Epoch: 135 	Training Loss: 0.017798 	Validation Loss: 0.308992
Epoch: 136 	Training Loss: 0.011208 	Validation Loss: 0.296819
Epoch: 137 	Training Loss: 0.021419 	Validation Loss: 0.265148
Epoch: 138 	Training Loss: 0.033870 	Validation Loss: 0.305339
Epoch: 139 	Training Loss: 0.028752 	Validation Loss: 0.299523
Epoch: 140 	Training Loss: 0.014696 	Validation Loss: 0.224561
Epoch: 141 	Training Loss: 0.007822 	Validation Loss: 0.198419
Epoch: 142 	Training Loss: 0.025131 	Validation Loss: 0.299411
Epoch: 143 	Training Loss: 0.025685 	Validation Loss: 0.320286
Epoch: 144 	Training Loss: 0.016572 	Validation Loss: 0.275005
Epoch: 145 	Training Loss: 0.023258 	Validation Loss: 0.252537
Epoch: 146 	Training Loss: 0.017674 	Validation Loss: 0.277172
Epoch: 147 	Training Loss: 0.017977 	Validation Loss: 0.276912
Epoch: 148 	Training Loss: 0.019551 	Validation Loss: 0.365398
Epoch: 149 	Training Loss: 0.035832 	Validation Loss: 0.299605
Epoch: 150 	Training Loss: 0.023161 	Validation Loss: 0.348122
Epoch: 151 	Training Loss: 0.023812 	Validation Loss: 0.346728

## ResNet-50
poch: 1 	Training Loss: 4.178554 	Validation Loss: 3.425520
Validation loss decreased (inf --> 3.425520).  Saving model ...
Epoch: 2 	Training Loss: 3.276455 	Validation Loss: 3.090532
Validation loss decreased (3.425520 --> 3.090532).  Saving model ...
Epoch: 3 	Training Loss: 2.694314 	Validation Loss: 2.777505
Validation loss decreased (3.090532 --> 2.777505).  Saving model ...
Epoch: 4 	Training Loss: 2.182725 	Validation Loss: 2.155407
Validation loss decreased (2.777505 --> 2.155407).  Saving model ...
Epoch: 5 	Training Loss: 1.758069 	Validation Loss: 1.402767
Validation loss decreased (2.155407 --> 1.402767).  Saving model ...
Epoch: 6 	Training Loss: 1.396181 	Validation Loss: 1.361954
Validation loss decreased (1.402767 --> 1.361954).  Saving model ...
Epoch: 7 	Training Loss: 1.126037 	Validation Loss: 1.009496
Validation loss decreased (1.361954 --> 1.009496).  Saving model ...
Epoch: 8 	Training Loss: 0.920513 	Validation Loss: 0.819674
Validation loss decreased (1.009496 --> 0.819674).  Saving model ...
Epoch: 9 	Training Loss: 0.825062 	Validation Loss: 0.981035
Epoch: 10 	Training Loss: 0.695655 	Validation Loss: 0.807769
Validation loss decreased (0.819674 --> 0.807769).  Saving model ...
Epoch: 11 	Training Loss: 0.593636 	Validation Loss: 0.646383
Validation loss decreased (0.807769 --> 0.646383).  Saving model ...
Epoch: 12 	Training Loss: 0.502310 	Validation Loss: 0.648955
Epoch: 13 	Training Loss: 0.458271 	Validation Loss: 0.550101
Validation loss decreased (0.646383 --> 0.550101).  Saving model ...
Epoch: 14 	Training Loss: 0.425872 	Validation Loss: 0.861265
Epoch: 15 	Training Loss: 0.357738 	Validation Loss: 0.600341
Epoch: 16 	Training Loss: 0.334991 	Validation Loss: 0.737613
Epoch: 17 	Training Loss: 0.319420 	Validation Loss: 0.548261
Validation loss decreased (0.550101 --> 0.548261).  Saving model ...
Epoch: 18 	Training Loss: 0.287515 	Validation Loss: 0.546595
Validation loss decreased (0.548261 --> 0.546595).  Saving model ...
Epoch: 19 	Training Loss: 0.252558 	Validation Loss: 0.505900
Validation loss decreased (0.546595 --> 0.505900).  Saving model ...
Epoch: 20 	Training Loss: 0.236396 	Validation Loss: 0.558215
Epoch: 21 	Training Loss: 0.216624 	Validation Loss: 0.495969
Validation loss decreased (0.505900 --> 0.495969).  Saving model ...
Epoch: 22 	Training Loss: 0.211475 	Validation Loss: 0.509249
Epoch: 23 	Training Loss: 0.194624 	Validation Loss: 0.620005
Epoch: 24 	Training Loss: 0.176580 	Validation Loss: 0.495104
Validation loss decreased (0.495969 --> 0.495104).  Saving model ...
Epoch: 25 	Training Loss: 0.168140 	Validation Loss: 0.509749
Epoch: 26 	Training Loss: 0.158852 	Validation Loss: 0.606116
Epoch: 27 	Training Loss: 0.142993 	Validation Loss: 0.506015
Epoch: 28 	Training Loss: 0.137268 	Validation Loss: 0.474988
Validation loss decreased (0.495104 --> 0.474988).  Saving model ...
Epoch: 29 	Training Loss: 0.138947 	Validation Loss: 0.511355
Epoch: 30 	Training Loss: 0.145748 	Validation Loss: 0.422663
Validation loss decreased (0.474988 --> 0.422663).  Saving model ...
Epoch: 31 	Training Loss: 0.107201 	Validation Loss: 0.625580
Epoch: 32 	Training Loss: 0.123979 	Validation Loss: 0.609367
Epoch: 33 	Training Loss: 0.141307 	Validation Loss: 0.535776
Epoch: 34 	Training Loss: 0.104338 	Validation Loss: 0.430829
Epoch: 35 	Training Loss: 0.108940 	Validation Loss: 0.471527
Epoch: 36 	Training Loss: 0.105201 	Validation Loss: 0.398817
Validation loss decreased (0.422663 --> 0.398817).  Saving model ...
Epoch: 37 	Training Loss: 0.107801 	Validation Loss: 0.464027
Epoch: 38 	Training Loss: 0.112246 	Validation Loss: 0.518139
Epoch: 39 	Training Loss: 0.097090 	Validation Loss: 0.465899
Epoch: 40 	Training Loss: 0.090036 	Validation Loss: 0.471108
Epoch: 41 	Training Loss: 0.088620 	Validation Loss: 0.413683
Epoch: 42 	Training Loss: 0.081320 	Validation Loss: 0.432726
Epoch: 43 	Training Loss: 0.092995 	Validation Loss: 0.450904
Epoch: 44 	Training Loss: 0.097590 	Validation Loss: 0.638215
Epoch: 45 	Training Loss: 0.077172 	Validation Loss: 0.471200
Epoch: 46 	Training Loss: 0.074464 	Validation Loss: 0.491423
Epoch: 47 	Training Loss: 0.057136 	Validation Loss: 0.457953
Epoch: 48 	Training Loss: 0.098887 	Validation Loss: 0.526458
Epoch: 49 	Training Loss: 0.062728 	Validation Loss: 0.452827
Epoch: 50 	Training Loss: 0.076807 	Validation Loss: 0.436726
Epoch: 51 	Training Loss: 0.073699 	Validation Loss: 0.469346
Epoch: 52 	Training Loss: 0.059821 	Validation Loss: 0.556752
Epoch: 53 	Training Loss: 0.055218 	Validation Loss: 0.405099
Epoch: 54 	Training Loss: 0.072779 	Validation Loss: 0.502286
Epoch: 55 	Training Loss: 0.063005 	Validation Loss: 0.384922
Validation loss decreased (0.398817 --> 0.384922).  Saving model ...
Epoch: 56 	Training Loss: 0.057353 	Validation Loss: 0.443255
Epoch: 57 	Training Loss: 0.063798 	Validation Loss: 0.399793
Epoch: 58 	Training Loss: 0.062251 	Validation Loss: 0.446099
Epoch: 59 	Training Loss: 0.059064 	Validation Loss: 0.491420
Epoch: 60 	Training Loss: 0.048233 	Validation Loss: 0.510500
Epoch: 61 	Training Loss: 0.062509 	Validation Loss: 0.415153
Epoch: 62 	Training Loss: 0.064723 	Validation Loss: 0.509590
Epoch: 63 	Training Loss: 0.055420 	Validation Loss: 0.479601
Epoch: 64 	Training Loss: 0.038810 	Validation Loss: 0.443790
Epoch: 65 	Training Loss: 0.044016 	Validation Loss: 0.516366
Epoch: 66 	Training Loss: 0.066071 	Validation Loss: 0.453642
Epoch: 67 	Training Loss: 0.048966 	Validation Loss: 0.366299
Validation loss decreased (0.384922 --> 0.366299).  Saving model ...
Epoch: 68 	Training Loss: 0.065516 	Validation Loss: 0.342470
Validation loss decreased (0.366299 --> 0.342470).  Saving model ...
Epoch: 69 	Training Loss: 0.037606 	Validation Loss: 0.461347
Epoch: 70 	Training Loss: 0.042595 	Validation Loss: 0.465020
Epoch: 71 	Training Loss: 0.040366 	Validation Loss: 0.515632
Epoch: 72 	Training Loss: 0.037161 	Validation Loss: 0.405927
Epoch: 73 	Training Loss: 0.054244 	Validation Loss: 0.363347
Epoch: 74 	Training Loss: 0.044851 	Validation Loss: 0.438505
Epoch: 75 	Training Loss: 0.045652 	Validation Loss: 0.378133
Epoch: 76 	Training Loss: 0.031520 	Validation Loss: 0.534948
Epoch: 77 	Training Loss: 0.060172 	Validation Loss: 0.455812
Epoch: 78 	Training Loss: 0.051409 	Validation Loss: 0.357653
Epoch: 79 	Training Loss: 0.045595 	Validation Loss: 0.448725
Epoch: 80 	Training Loss: 0.033657 	Validation Loss: 0.393312
Epoch: 81 	Training Loss: 0.032830 	Validation Loss: 0.368492
Epoch: 82 	Training Loss: 0.034815 	Validation Loss: 0.408718
Epoch: 83 	Training Loss: 0.057467 	Validation Loss: 0.437323
Epoch: 84 	Training Loss: 0.043768 	Validation Loss: 0.342392
Validation loss decreased (0.342470 --> 0.342392).  Saving model ...
Epoch: 85 	Training Loss: 0.023519 	Validation Loss: 0.369158
Epoch: 86 	Training Loss: 0.033114 	Validation Loss: 0.396901
Epoch: 87 	Training Loss: 0.036653 	Validation Loss: 0.408922
Epoch: 88 	Training Loss: 0.042422 	Validation Loss: 0.418844
Epoch: 89 	Training Loss: 0.051546 	Validation Loss: 0.392979
Epoch: 90 	Training Loss: 0.034048 	Validation Loss: 0.418823
Epoch: 91 	Training Loss: 0.022448 	Validation Loss: 0.336940
Validation loss decreased (0.342392 --> 0.336940).  Saving model ...
Epoch: 92 	Training Loss: 0.043167 	Validation Loss: 0.419201
Epoch: 93 	Training Loss: 0.028628 	Validation Loss: 0.392072
Epoch: 94 	Training Loss: 0.040354 	Validation Loss: 0.477884
Epoch: 95 	Training Loss: 0.039656 	Validation Loss: 0.448052
Epoch: 96 	Training Loss: 0.023276 	Validation Loss: 0.387998
Epoch: 97 	Training Loss: 0.025297 	Validation Loss: 0.399453
Epoch: 98 	Training Loss: 0.045902 	Validation Loss: 0.464934
Epoch: 99 	Training Loss: 0.021254 	Validation Loss: 0.403342
Epoch: 100 	Training Loss: 0.036754 	Validation Loss: 0.408717

## Deeper Dense 121

### (the kernel froze and most of the logging was lost :( over 75 epochs completed w/ the best at epoch 52)

Epoch: 1 	Training Loss: 4.240837 	Validation Loss: 3.848322
Validation loss decreased (inf --> 3.848322).  Saving model ...
Epoch: 2 	Training Loss: 3.830423 	Validation Loss: 3.414813
Validation loss decreased (3.848322 --> 3.414813).  Saving model ...
Epoch: 3 	Training Loss: 3.387042 	Validation Loss: 2.832042
Validation loss decreased (3.414813 --> 2.832042).  Saving model ...
Epoch: 4 	Training Loss: 3.031815 	Validation Loss: 2.807809
Validation loss decreased (2.832042 --> 2.807809).  Saving model ...
Epoch: 5 	Training Loss: 2.810778 	Validation Loss: 2.035235
Validation loss decreased (2.807809 --> 2.035235).  Saving model ...
Epoch: 6 	Training Loss: 2.542110 	Validation Loss: 1.708808
Validation loss decreased (2.035235 --> 1.708808).  Saving model ...
Epoch: 7 	Training Loss: 2.339306 	Validation Loss: 1.512875
Validation loss decreased (1.708808 --> 1.512875).  Saving model ...
Epoch: 8 	Training Loss: 2.204065 	Validation Loss: 1.429649
Validation loss decreased (1.512875 --> 1.429649).  Saving model ...
Epoch: 9 	Training Loss: 2.071992 	Validation Loss: 1.320966
Validation loss decreased (1.429649 --> 1.320966).  Saving model ...
Epoch: 10 	Training Loss: 1.971295 	Validation Loss: 1.285134
Validation loss decreased (1.320966 --> 1.285134).  Saving model ...
Epoch: 11 	Training Loss: 1.918845 	Validation Loss: 1.188760
Validation loss decreased (1.285134 --> 1.188760).  Saving model ...
Epoch: 12 	Training Loss: 1.821173 	Validation Loss: 1.012195
Validation loss decreased (1.188760 --> 1.012195).  Saving model ...
Epoch: 13 	Training Loss: 1.736375 	Validation Loss: 0.973106
Validation loss decreased (1.012195 --> 0.973106).  Saving model ...
Epoch: 14 	Training Loss: 1.664966 	Validation Loss: 0.899205
Validation loss decreased (0.973106 --> 0.899205).  Saving model ...
Epoch: 15 	Training Loss: 1.580073 	Validation Loss: 0.846053
Validation loss decreased (0.899205 --> 0.846053).  Saving model ...
Epoch: 16 	Training Loss: 1.575959 	Validation Loss: 0.796266
Validation loss decreased (0.846053 --> 0.796266).  Saving model ...
Epoch: 17 	Training Loss: 1.554354 	Validation Loss: 0.741671
Validation loss decreased (0.796266 --> 0.741671).  Saving model ...
Epoch: 18 	Training Loss: 1.462975 	Validation Loss: 0.695325
Validation loss decreased (0.741671 --> 0.695325).  Saving model ...
Epoch: 19 	Training Loss: 1.468491 	Validation Loss: 0.713486
Epoch: 20 	Training Loss: 1.400486 	Validation Loss: 0.686692
Validation loss decreased (0.695325 --> 0.686692).  Saving model ...
Epoch: 21 	Training Loss: 1.368541 	Validation Loss: 0.661797
Validation loss decreased (0.686692 --> 0.661797).  Saving model ...
Epoch: 22 	Training Loss: 1.327788 	Validation Loss: 0.636063
Validation loss decreased (0.661797 --> 0.636063).  Saving model ...
Epoch: 23 	Training Loss: 1.315603 	Validation Loss: 0.691996
Epoch: 24 	Training Loss: 1.304906 	Validation Loss: 0.602535
Validation loss decreased (0.636063 --> 0.602535).  Saving model ...
Epoch: 25 	Training Loss: 1.253940 	Validation Loss: 0.667941
Epoch: 26 	Training Loss: 1.228192 	Validation Loss: 0.603391
Epoch: 27 	Training Loss: 1.203728 	Validation Loss: 0.598728
Validation loss decreased (0.602535 --> 0.598728).  Saving model ...
Epoch: 28 	Training Loss: 1.184133 	Validation Loss: 0.562698
Validation loss decreased (0.598728 --> 0.562698).  Saving model ...